## 1. Introduction <a id="1"></a>

The [Video Game Sales](https://www.kaggle.com/gregorut/videogamesales) dataset provides an interesting way to assess most popular games through the lense of the total amount of sales made. But what if we wanted to look at collections of similar games, i.e., game franchises, rather than individual games? There is no unique ID to identify a franchise and since the games in the same franchise will have slightly different names (e.g. Pokémon Red, Pokémon Gold, Pokémon Crystal, etc.) you cannot simply use the groupby method on them.

This notebook explores how the [Python Record Linkage Toolkit](https://pypi.org/project/recordlinkage/) can help us in this task by grouping similar name games together to idetify potential franchises.

<a><img src="https://images.unsplash.com/photo-1566577134648-41d15b958d80?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1650&q=80"></a> <p style="text-align:center">*In 2020, the Final Fantasy game franchise consists of 15 titles*</p>


### **Table of Contents**
* [Introduction](#1)
* [Setup](#2)
* [Data Exploration](#3)
* [Indexing](#4)
* [Pair Comparison](#5)
* [Testing the Results](#6)
* [Final Considerations](#7)

## Setup <a id="2"></a>

In [1]:
# Install the Record Linkage library (it is not part of the standard Kaggle notebook environment)
%pip install recordlinkage

# Import libraries
import pandas as pd

import plotly.express as px

import recordlinkage as rl

import networkx as nx

# Set notebook properties
pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 100

# Import the data
df = pd.read_csv('../input/videogamesales/vgsales.csv')

print(df.shape)
df.head()

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
(16598, 11)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.00,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.00,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.00,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.00,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.00,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [2]:
# Make all column names lower for ease of typing
df.columns = df.columns.str.lower()

In [3]:
# Inspect the dataset holistically 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rank          16598 non-null  int64  
 1   name          16598 non-null  object 
 2   platform      16598 non-null  object 
 3   year          16327 non-null  float64
 4   genre         16598 non-null  object 
 5   publisher     16540 non-null  object 
 6   na_sales      16598 non-null  float64
 7   eu_sales      16598 non-null  float64
 8   jp_sales      16598 non-null  float64
 9   other_sales   16598 non-null  float64
 10  global_sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


There are missing values in the `year` and `publisher` variables. Although not ideal, this will not impact further analysis, so we can leave them blank.

In [4]:
# Inspect the numeric variables
df.describe()

,rank,year,na_sales,eu_sales,jp_sales,other_sales,global_sales
count,16598.00,16327.00,16598.00,16598.00,16598.00,16598.00,16598.00
mean,8300.61,2006.41,0.26,0.15,0.08,0.05,0.54
std,4791.85,5.83,0.82,0.51,0.31,0.19,1.56
min,1.00,1980.00,0.00,0.00,0.00,0.00,0.01
25%,4151.25,2003.00,0.00,0.00,0.00,0.00,0.06
50%,8300.50,2007.00,0.08,0.02,0.00,0.01,0.17
75%,12449.75,2010.00,0.24,0.11,0.04,0.04,0.47
max,16600.00,2020.00,41.49,29.02,10.22,10.57,82.74


Nothing particularly stands out here, so we can move forward.

## Data Exploration <a id="3"></a>

In this section we will explore the data variables in a bit more detail. This will give us a better understanding of the overall context.

In [5]:
# Print unique values of the relevant columns 
print(f'Unique games in the dataset: {df.name.nunique()}')
print(f'Unique publishers in the dataset: {df.publisher.nunique()}')
print(f'Unique platforms in the dataset: {df.platform.nunique()}')
print(f'Unique genres in the dataset: {df.genre.nunique()}')

Unique games in the dataset: 11493
Unique publishers in the dataset: 578
Unique platforms in the dataset: 31
Unique genres in the dataset: 12


There are around 11.5K unique game titles in the dataset. This will be the number that we will try to reduce by grouping the games into franchises.

In [6]:
# Check if names are duplicated
df.groupby(['name'])['name'].count().sort_values(ascending=False)

name
Need for Speed: Most Wanted                    12
FIFA 14                                         9
Madden NFL 07                                   9
LEGO Marvel Super Heroes                        9
Ratatouille                                     9
                                               ..
PenPen TriIceLon                                1
Pen 1 Grand Prix: Penguin no Mondai Special     1
Peggle: Dual Shot                               1
Peggle Nights                                   1
'98 Koshien                                     1
Name: name, Length: 11493, dtype: int64

In [7]:
# Inspect one of the duplicated names
df[df['name'] == 'Need for Speed: Most Wanted']

,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
252,253,Need for Speed: Most Wanted,PS2,2005.00,Racing,Electronic Arts,2.03,1.79,0.08,0.47,4.37
498,499,Need for Speed: Most Wanted,PS3,2012.00,Racing,Electronic Arts,0.71,1.51,0.06,0.61,2.89
1173,1175,Need for Speed: Most Wanted,X360,2012.00,Racing,Electronic Arts,0.62,0.80,0.01,0.15,1.58
1530,1532,Need for Speed: Most Wanted,X360,2005.00,Racing,Electronic Arts,1.00,0.17,0.02,0.10,1.29
1742,1744,Need for Speed: Most Wanted,PSV,2012.00,Racing,Electronic Arts,0.38,0.52,0.01,0.25,1.16
2005,2007,Need for Speed: Most Wanted,XB,2005.00,Racing,Electronic Arts,0.53,0.46,0.00,0.05,1.04
3585,3587,Need for Speed: Most Wanted,GC,2005.00,Racing,Electronic Arts,0.43,0.11,0.00,0.02,0.56
5900,5902,Need for Speed: Most Wanted,PC,2005.00,Racing,Electronic Arts,0.02,0.23,0.00,0.05,0.30
6149,6151,Need for Speed: Most Wanted,WiiU,2013.00,Racing,Electronic Arts,0.14,0.12,0.00,0.02,0.28
6278,6280,Need for Speed: Most Wanted,DS,2005.00,Racing,Electronic Arts,0.24,0.01,0.00,0.02,0.27


Duplicates exist. This is because the same game can be released on multiple platforms and rereleased in different years. However, we do not need to deduplicate, as we want to see the total sales for each franchise regardless of platform or release year.

In [8]:
fig = px.bar(df.sort_values(by=['global_sales'],ascending=False)[:20], x='name', y='global_sales')
fig.show()

In [9]:
by_year = df.groupby(['name', 'year'], as_index=False)['global_sales'].sum()
by_year = by_year[by_year['year'] <= 2016]
by_year = by_year.sort_values(by=['year','global_sales'],ascending=False)

fig = px.bar(by_year, x="name", y="global_sales", animation_frame="year", hover_name="name")

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

A few observations:
* Data for 2016 seems to be incomplete.
* Wii Sports is the most sold game in history by a far. This is particularly amazing considering that Wii Sport has been released on multiple  gaming platforms.

## Indexing <a id="4"></a>

Now, we will begin the process of matching the games.

To assess if two games are related, we need to compare them. Since we have 11,493 titles in the dataset, we would have to create 66,038,776 comparison pairs (11,493 x 11,492 / 2). Even if one pair takes only 1 second to compare, it would take approximately the amount of time it took to develop Dark Souls (around 2 years) to compute.

One way to reduce this complexity is to limit the number of comparable pairs by assuming that most related games will have similar names. This naturally makes sense, as once a studio has a well selling game, it will maintain a similar name for all the subsequent releases to make the franchise more recognisable and thus potentially increasing the number of sales while reducing marketing costs.

The approach that the Record Linkage library utilises for limiting the number of pairs is called Sorted Neighbourhood Indexing. It is a fairly simple method that follows these steps:

1. Sorts the entire dataset in alphabetic order.
2. Divides  the dataset into small windows of words.
3. Creates pairs of words within each window that can be assessed. 

In [10]:
df_rl = df.set_index(['name'], drop=False)
df_rl = df_rl.drop_duplicates(subset=['name'])

indexer = rl.SortedNeighbourhoodIndex('name', window=3)
candidate_pairs = indexer.index(df_rl)

len(candidate_pairs)

11492

In [11]:
print(df.shape)
print(df_rl.shape)
display(df.head(2))
print(candidate_pairs[0])

(16598, 11)
(11493, 11)


,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006.00,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.00,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24


('Mario Kart DS', 'Mario Kart Wii')


Running the Sorted Neighbourhood Indexing generated 45962 candidate pairs for us to assess. 

## Pair Comparison <a id="5"></a>

Now that we have the candidate pairs, we need to assess if those pairs are actual matches (belong to the same franchise) or not.

Record Linkage allows to choose from multiple similarity metrics. The one that works well with short strings like names is called [Jaro-Winkler](https://statisticaloddsandends.wordpress.com/2019/09/11/what-is-jaro-jaro-winkler-similarity/). It assigns a numeric similarity score between 0 and 1 for any two strings based on:
* The number of matching characters
* Number of transpositions
* If the differences occur at the beginning or the end of the string

In addition, the library allows to compare additional variables. For example, here we assign values of 1 where the platform and the genre of the game match.

In [12]:
compare_cp = rl.Compare()

compare_cp.string('name','name',method='jarowinkler',label='name') # Could use the threshold parameter
compare_cp.exact('platform','platform',label='platform')
compare_cp.exact('genre','genre',label='genre')

features = compare_cp.compute(candidate_pairs, df_rl)

In [13]:
features.head()

,,name,platform,genre
name_1,name_2,,,
Mario Kart DS,Mario Kart Wii,0.93,0,1
Call of Duty: Modern Warfare 2,Call of Duty: Modern Warfare 3,0.99,1,1
Grand Theft Auto IV,Grand Theft Auto V,0.99,0,1
Pokemon FireRed/Pokemon LeafGreen,Pokemon Gold/Pokemon Silver,0.83,0,1
Call of Duty: Ghosts,Call of Duty: Modern Warfare 2,0.85,1,1


We have our results! Now, we need a way to assess if the pairs are actual matches or should be discarded. We can do this multiple ways:
* Simply use the Jaro-Winkler metric alone.
* Use the Jaro-Winkler metric plus a combination of the other values. 
* If we manually inspected and labelled the data, we could actually create a machine learning model that would help us pick the best combination of variables and their values to determine matches.

In [14]:
column_list = ['platform','genre']

matches = features[(features['name'] > 0.85)]

#matches = features[(features['name'] > 0.85) & (features[column_list].sum(axis=1) >= 1)]

For now, we will simply use an arbitrary threshold of the Jaro-Winkler similarity.

To move from pairs to groups we can utalise the NetworkX library. It will allows us to efficiently connect the individual pairs and genererate a list of sets with the games grouped. 

In [15]:
l = matches.index.tolist()

G = nx.Graph()

G.add_edges_from(l)

groups = list(nx.connected_components(G))

groups[:5]

[{'Mario Kart 64',
  'Mario Kart 7',
  'Mario Kart 8',
  'Mario Kart DS',
  'Mario Kart Wii'},
 {'Call of Duty: Modern Warfare 2',
  'Call of Duty: Modern Warfare 3',
  'Call of Duty: Modern Warfare 3: Defiance',
  'Call of Duty: Modern Warfare Trilogy',
  'Call of Duty: Modern Warfare: Mobilized',
  'Call of Duty: Modern Warfare: Reflex Edition'},
 {'Grand Theft Auto',
  'Grand Theft Auto 2',
  'Grand Theft Auto III',
  'Grand Theft Auto IV',
  'Grand Theft Auto V',
  'Grand Theft Auto: Chinatown Wars',
  'Grand Theft Auto: Liberty City Stories'},
 {'Gran Turismo',
  'Gran Turismo (PSP)',
  'Gran Turismo 2',
  'Gran Turismo 3: A-Spec',
  'Gran Turismo 4',
  'Gran Turismo 4 Prologue',
  'Gran Turismo 5',
  'Gran Turismo 5 Prologue',
  'Gran Turismo 6',
  'Gran Turismo Concept 2001 Tokyo'},
 {'Animal Crossing',
  'Animal Crossing: Amiibo Festival',
  'Animal Crossing: City Folk',
  'Animal Crossing: Happy Home Designer',
  'Animal Crossing: New Leaf',
  'Animal Crossing: Wild World'}]

The code seems to be working! Alhough some false positives can be seen. 

In [16]:
group_ids = {name:k for k,comp in enumerate(groups) for name in comp}

groups_df = pd.DataFrame.from_dict(group_ids, orient='index', columns=['group']).rename_axis('name').reset_index()

In [17]:
df_rl = df_rl.reset_index(drop=True)

merged = df_rl.merge(groups_df, on='name', how='left')

merged['group'] = merged['group'].fillna(merged['name'])

In [18]:
merged.head(3)

,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,group
0,1,Wii Sports,Wii,2006.00,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,309.00
1,2,Super Mario Bros.,NES,1985.00,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,9.00
2,3,Mario Kart Wii,Wii,2008.00,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.00


In [19]:
top_sales = merged.groupby(['group', 'name'], as_index=False)['global_sales'].max()
top_sales = top_sales.sort_values(by='global_sales', ascending=False)
top_sales = top_sales.drop_duplicates(subset=['group'], keep='first')

del top_sales['global_sales']
top_sales = top_sales.rename(columns={'name':'name_rl'})

df_rl = merged.merge(top_sales, on='group', how='left')

## Testing the Results <a id="6"></a>

In [20]:
df_rl.name_rl.nunique()

6619

In [21]:
top_sales.head(20)

,group,name_rl
1932,309.00,Wii Sports
57,9.00,Super Mario Bros.
4,0.00,Mario Kart Wii
5159,1212.00,Pokemon Red/Pokemon Blue
989,145.00,Tetris
130,20.00,New Super Mario Bros.
504,71.00,Wii Play
7999,Duck Hunt,Duck Hunt
5170,1217.00,Nintendogs
5161,1213.00,Pokemon Gold/Pokemon Silver


Looks like we managed to reduce the number of unique games by more than 50%!

## Final Considerations <a id="7"></a>

Overall, we can see that the approach is effective in finding matching names with the overall reduction of unique titles in the dataset by almost 50%.

The same library and approach discussed above is domain agnostic, so it can be applied to a wide variety of projects. Furthermore, it can be used on a variety of related problems like finding duplicated records in a dataset or finding duplicates when merging two datasets.

However, it is worth to note that the method has its flaws:
1. It assumes that games in the same franchise will have similar names. While this assumption is usually correct based on the logic discussed in the indexing step, there are exceptions. Consider the following games in the Star Wars universe: [X-Wing](https://en.wikipedia.org/wiki/Star_Wars:_X-Wing), [Clone Wars Adventures](https://en.wikipedia.org/wiki/Clone_Wars_Adventures), [Vader Immortal](https://en.wikipedia.org/wiki/Darth_Vader#Virtual_reality_game). The names have no relation to each other, and the approach discussed would not be able to group them.

2. There is no effective way to assess the quality of the results other than manually inspecting them.

It is also worth to note that Python Record Linkage Toolkit is not the only library that exits for the purpose of matching entities. Two other popular libraries include: Dedupe and Fuzzywuzzy.